# Gemini

In [1]:
from utils.model_setup import *
from utils.dataset_setup import *
from utils.experiment_setup import *
from utils.save_results import *
from utils.state_control import *

In [2]:
from importlib import reload

In [4]:
path_to = "./data/"
path_to_experiments = "./experiments/"
state_filename = 'EstadoExperimentosChat_gemini.csv'
model_identifier = 'gemini'

## Excuting experiments

In [5]:
def run_experiment(exp, essays_dataset, start, limit, batch, path_to_save, filename_to_save, model_name, chat, state, path_to):
    end = start
    for ini in range(start, limit-batch, batch):
        end = ini + batch
        try:
            essays_outputs = run_model(essays_dataset, exp, ini, end, model_name, chat)
        except Exception as e:
            print(f"There was an exception on run_experiment!!: {e}")
            time.sleep(600)
            chat = model_setup(model_name)
            essays_outputs = run_model(essays_dataset, exp, ini, end, run_model, chat)
        
        if ini == 0:
            create_csv_header(path_to_save, filename_to_save, essays_outputs[0].keys())
        
        save_data(path_to_save, filename_to_save, essays_outputs)
        
        print("Processed essays from " + str(ini) + " to: " + str(end))
        new_state = {
            'Batch': end
        }
        state = update_state(state, new_state)
        save_state(state, path_to, model_identifier)

    print("End value is: ", end)
    print("Limit value is:", limit)

    if limit !=  end:
        essays_outputs = run_model(essays_dataset, exp, end, limit, model_name, chat)
        save_data(path_to_save, filename_to_save, essays_outputs)
        print("Processed essays from " + str(end) + " to: " + str(limit))

## Setup

In [6]:
use_experiment_state = True

state = create_state(-1, -1, -1, 1, 5, 0)
if use_experiment_state:
  state = retrieve_state(path_to, model_identifier)

{'Model': 1, 'Database': 2, 'Experimento': 5, 'Execucao': 2, 'MaxExecucao': 5, 'Batch': 130}


In [7]:
print(state)

{'Model': 1, 'Database': 2, 'Experimento': 5, 'Execucao': 2, 'MaxExecucao': 5, 'Batch': 130}


In [8]:
"""
Setup Model
"""
model_name, model_code = choose_model(state['Model'])
chat = model_setup(model_name)

Choosen Model: gemini-1.5-flash


KeyError: 'GOOGLE_API_KEY_1'

In [38]:
"""
Setup Dataset
"""
dataset_name, dataset_code = setup_dataset(state['Database'])
essays_dataset = getDataset(path_to, dataset_name)

Dataset escolhido: aes_enem_dataset


In [42]:
while state['Experimento'] <= 6:
    """
    Setup Experiment
    """
    experiment_name, experiment_code = choose_experiment(state['Experimento'])
    exp = setup_experiment(path_to_experiments, experiment_name)
    
    new_udpates = {
        'Model': model_code,
        'Database': dataset_code,
        'Experimento': experiment_code,
    }
    print(state)
    state = update_state(state, new_udpates)
    print(state)
    save_state(state, path_to, model_identifier)
    
    while state['Execucao'] <= state['MaxExecucao']:
        print()
        print()
        print("Execution number: " + str(state['Execucao']))
        print()
        print()
        
        start = state['Batch']
        path_to_save = create_experiment_folder(path_to, model_name, experiment_name, dataset_name)
        filename_to_save = build_filename_to_save(model_name, experiment_name, dataset_name, str(state['Execucao']))
        
        batch = 10
        limit = len(essays_dataset)
        
        run_experiment(exp, essays_dataset, start, limit, batch, path_to_save, filename_to_save, model_name, chat, stat, path_to)
        
        '''
        Change to new execution
        '''
        new_state = {
            'Execucao': state['Execucao'] + 1,
            'Batch': 0
        }
        state = update_state(state, new_state)
        save_state(state, path_to, model_identifier)

    '''
    Change to new experiment
    '''
    new_state = {
        'Execucao': 0,
        'Experimento': state['Experimento'] + 1,
        'Batch': 0
    }
    state = update_state(state, new_state)
    save_state(state, path_to, model_identifier)


Experimento escolhido: exp5
Create exp5
Prompt template: 
Qual o método de avaliação utilizado na correção de redações do ENEM?

####

Corrija a redação abaixo seguindo o método de avaliação do ENEM. Atribuindo notas quantitativas entre 0 e 200 para cada uma das cinco competências. Forneça uma explicação detalhada que justifique cada nota.


Tema:
"""
<TEMA>
"""


Redação:
"""
<REDAÇÃO>
"""

####

Com os dados da reposta anterior, forneça apenas as notas quantitativas entre 0 e 200 para cada competência, e a nota final, no seguinte formato:[C1,C2,C3,C4,C5,Total], onde "Total" é a soma das notas das cinco competências. 

Não forneça texto na resposta, apenas as notas numéricas para competência e o total.
{'Model': 1, 'Database': 2, 'Experimento': 5, 'Execucao': 2, 'MaxExecucao': 5, 'Batch': 100}
{'Model': 1, 'Database': 2, 'Experimento': 5, 'Execucao': 2, 'MaxExecucao': 5, 'Batch': 100}
Saving state: {'Model': 1, 'Database': 2, 'Experimento': 5, 'Execucao': 2, 'MaxExecucao': 5, 'Batch':

KeyboardInterrupt: 

In [ ]:
"""
Setup Experiment
"""
experiment_name, experiment_code = choose_experiment(experiment_code)
exp = setup_experiment(path_to_experiments, experiment_name)

In [ ]:
essay_data = essays_dataset[2]
essay = essay_data["essay_text"]
prompt = essay_data["id_prompt"].replace("-", " ")
prompt = exp.build_prompt(essay, prompt)

model_prompt = exp.build_prompt(essay, prompt)

In [ ]:
grades, response = made_question(model_name, chat, model_prompt)

In [ ]:
print